In [1]:
#@markdown ## ⚠️ PLEASE NOTE:
#@markdown This colab runs best using a GPU runtime.  From the Colab menu, choose Runtime > Change Runtime Type, then select **'GPU'** in the dropdown.

import functools
import jax
import os

from datetime import datetime
from jax import numpy as jp
import matplotlib.pyplot as plt

from IPython.display import HTML, clear_output

try:
  !pip uninstall -y brax
  !pip install git+https://github.com/izo0619/brax.git@mod_hopper
  # !pip install git+https://github.com/google/brax.git@main
  import brax
  # clear_output()
except ImportError:
  #!pip install git+https://github.com/google/brax.git@main
  !pip install git+https://github.com/izo0619/brax.git@three_legs
  clear_output()
  import brax

import flax
from brax import envs
from brax.io import model
from brax.io import json
from brax.io import html
from brax.training.agents.ppo import train as ppo
from brax.training.agents.sac import train as sac

Found existing installation: brax 0.9.3
Uninstalling brax-0.9.3:
  Successfully uninstalled brax-0.9.3
  Cloning https://github.com/izo0619/brax.git (to revision mod_hopper) to /private/var/folders/_5/gr45nmk51ps_bl3hxbzqxczc0000gn/T/pip-req-build-sqxxdcqn
  Running command git clone --filter=blob:none --quiet https://github.com/izo0619/brax.git /private/var/folders/_5/gr45nmk51ps_bl3hxbzqxczc0000gn/T/pip-req-build-sqxxdcqn
  Running command git checkout -b mod_hopper --track origin/mod_hopper
  Switched to a new branch 'mod_hopper'
  Branch 'mod_hopper' set up to track remote branch 'mod_hopper' from 'origin'.
  Resolved https://github.com/izo0619/brax.git to commit 2e85f029d60a4d252d32f360b4531ba89da2178e
  Preparing metadata (setup.py) ... done
  Created wheel for brax: filename=brax-0.9.3-py3-none-any.whl size=5901328 sha256=25d2a30bdb6b32194d3fe0bb5c453c57af2b27d068147d3d4b6fb45995935a54
  Stored in directory: /private/var/folders/_5/gr45nmk51ps_bl3hxbzqxczc0000gn/T/pip-ephem-whee

In [2]:
#@title Load Env { run: "auto" }

env_name = 'links'  # @param ['swimmer','links', 'ant', 'halfcheetah', 'hopper', 'humanoid', 'humanoidstandup', 'inverted_pendulum', 'inverted_double_pendulum', 'pusher', 'reacher', 'walker2d']
backend = 'generalized'  # @param ['generalized', 'positional', 'spring']

env = envs.get_environment(env_name=env_name,
                           backend=backend)
state = jax.jit(env.reset)(rng=jax.random.PRNGKey(seed=0))
# print(state.pipeline_state)
# print(env.sys.geoms)
# print(html.render(env.sys, [state.pipeline_state]))
HTML(html.render(env.sys, [state.pipeline_state]))

In [3]:
# print(env.sys.geoms)
state = jax.jit(env.reset)(rng=jax.random.PRNGKey(seed=0))
# print(state.pipeline_state)
# state = jax.jit(env.step)(state=state, action=jp.array([3]))
# print(state.pipeline_state)
# print(state.pipeline_state)
HTML(html.render(env.sys, [state.pipeline_state]))

In [6]:
#@title Visualizing a trajectory of the learned inference function

# create an env with auto-reset
env = envs.create(env_name=env_name, backend=backend)

jit_env_reset = jax.jit(env.reset)
jit_env_step = jax.jit(env.step)

rollout = []
rng = jax.random.PRNGKey(seed=1)
state = jit_env_reset(rng=rng)
for _ in range(10):
  print(_)
  rollout.append(state.pipeline_state)
  act = jp.ones(env.sys.act_size())
  state = jit_env_step(state, act)
print(html.render(env.sys.replace(dt=env.dt), rollout))
HTML(html.render(env.sys.replace(dt=env.dt), rollout))

0
1
2
3
4
5
6
7
8
9
<!DOCTYPE html>
<html>

  <head>
    <title>Brax visualizer</title>
    <link rel="shortcut icon" type="image/x-icon" href="/favicon.ico">
  </head>

  <style>
    body {
      margin: 0;
      padding: 0;
    }

    #brax-viewer {
      height: 480px;
      margin: 0;
      padding: 0;
    }
  </style>
  <script async src="https://unpkg.com/es-module-shims@1.6.3/dist/es-module-shims.js"></script>

  <script type="importmap">
    {
      "imports": {
        "three": "https://unpkg.com/three@0.150.1/build/three.module.js",
        "three/addons/": "https://unpkg.com/three@0.150.1/examples/jsm/",
        "lilgui": "https://cdn.jsdelivr.net/npm/lil-gui@0.18.0/+esm",
        "viewer": "https://cdn.jsdelivr.net/gh/google/brax@v0.9.3/brax/visualizer/js/viewer.js"
      }
    }
  </script>

  <script type="application/javascript">
  var system = {"dt": 0.03999999910593033, "gravity": [0.0, 0.0, -9.8100004196167], "viscosity": 0.0, "density": 0.0, "link": {"transform": {"p